In [ ]:
# default_exp methods

# Hierachical Reconciliation Methods

Table of Contents
1.   [Bottom Up](#cell-1)
2.   [Top Down](#cell-2)
3.   [Min Trace](#cell-3)
4.   [Empirical Risk Minimization](#cell-4)

In [ ]:
#export
from typing import List

import numpy as np
from statsmodels.stats.moment_helpers import cov2corr

In [ ]:
#hide
from fastcore.test import ExceptionExpected, test_close, test_eq

In [ ]:
#export
def _reconcile(S: np.ndarray, P: np.ndarray, W: np.ndarray, 
               y_hat: np.ndarray, SP: np.ndarray = None):
    if SP is None:
        SP = S @ P
    return np.matmul(SP, y_hat)

<a name="cell-1"></a>
## 1. Bottom Up

The most basic hierarchical reconciliation is performed using an Bottom-Up strategy. It was proposed for the first time by Orcutt in 1968.
The corresponding hierarchical "projection" matrix is defined as:

$$\mathbf{P}_{\text{BU}} = [\mathbf{0}_{\mathrm{[b],[a]}}\;|\;\mathbf{I}_{\mathrm{[b][b]}}]$$

- [Orcutt, G.H., Watts, H.W., & Edwards, J.B.(1968). Data aggregation and information loss. The American Economic Review, 58 , 773{787)](http://www.jstor.org/stable/1815532)

In [ ]:
#export
def bottom_up(S: np.ndarray,
              y_hat: np.ndarray,
              idx_bottom: List[int]):
    n_hiers, n_bottom = S.shape
    P = np.zeros_like(S, dtype=np.float32)
    P[idx_bottom] = S[idx_bottom]
    P = P.T
    W = np.eye(n_hiers, dtype=np.float32)
    return _reconcile(S, P, W, y_hat)

In [ ]:
#export
class BottomUp:
    
    def reconcile(self,
                  S: np.ndarray,
                  y_hat: np.ndarray,
                  idx_bottom: np.ndarray):
        return bottom_up(S=S, y_hat=y_hat, idx_bottom=idx_bottom)
    
    __call__ = reconcile

In [ ]:
#hide
S = np.array([
    [1., 1., 1., 1.],
    [1., 1., 0., 0.],
    [0., 0., 1., 1.],
    [0., 1., 0., 0.],
    [1., 0., 0., 0.],
    [0., 0., 1., 0.],
    [0., 0., 0., 1.],
])
h = 10
_y = np.array([10., 5., 4., 2., 1.])
y_bottom = np.vstack([i * _y for i in range(1, 5)])
resids_bottom = y_bottom - np.roll(y_bottom, 1)
resids_bottom[:, 0] = np.nan
y_hat_bottom = np.vstack([i * np.ones(h) for i in range(1, 5)])
idx_bottom = [4, 3, 5, 6]

In [ ]:
#hide
cls_bottom_up = BottomUp()
test_eq(
    cls_bottom_up(S=S, y_hat=S @ y_hat_bottom, idx_bottom=idx_bottom),
    S @ y_hat_bottom
)

<a name="cell-2"></a>
## 2. Top Down

The Top Down hierarchical reconciliation method, distributes the total aggregate predictions and decomposes it down the hierarchy using proportions $\mathbf{p}_{\mathrm{[b]}}$ that can be actual historical values or estimated.

$$\mathbf{P}=[\mathbf{p}_{\mathrm{[b]}}\;|\;\mathbf{0}_{\mathrm{[b][a,b\;-1]}}]$$

- [Disaggregation methods to expedite product line forecasting. Journal of Forecasting, 9 , 233–254. doi:10.1002/for.3980090304](https://onlinelibrary.wiley.com/doi/abs/10.1002/for.3980090304)
- [An investigation of aggregate variable time series forecast strategies with specific subaggregate time series statistical correlation. Computers and Operations Research, 26 , 1133–1149. doi:10.1016/S0305-0548(99)00017-9](https://doi.org/10.1016/S0305-0548(99)00017-9)

In [ ]:
#export
def top_down(S: np.ndarray, 
             y_hat: np.ndarray,
             y: np.ndarray,
             idx_bottom: List[int],
             method: str):
    n_hiers, n_bottom = S.shape
    idx_top = int(S.sum(axis=1).argmax())
    #add strictly hierarchical assert
    
    if method == 'forecast_proportions':
        raise NotImplementedError(f'Method {method} not implemented yet')
    else:
        y_top = y[idx_top]
        y_btm = y[idx_bottom]
        if method == 'average_proportions':
            prop = np.mean(y_btm / y_top, axis=1)
        elif method == 'proportion_averages':
            prop = np.mean(y_btm, axis=1) / np.mean(y_top)
        else:
            raise Exception(f'Unknown method {method}')
    P = np.zeros_like(S).T
    P[:, idx_top] = prop
    W = np.eye(n_hiers, dtype=np.float32)
    return _reconcile(S, P, W, y_hat)

In [ ]:
#export
class TopDown:
    
    def __init__(self, method: str):
        self.method = method
    
    def reconcile(self, 
                  S: np.ndarray, 
                  y_hat: np.ndarray,
                  y: np.ndarray,
                  idx_bottom: List[int]):
        return top_down(S=S, y_hat=y_hat, y=y, 
                        idx_bottom=idx_bottom,
                        method=self.method)
    
    __call__ = reconcile

In [ ]:
#hide
for method in ['average_proportions', 'proportion_averages']:
    cls_top_down = TopDown(method=method)
    test_close(
        cls_top_down(
            S=S, 
            y_hat=S @ y_hat_bottom, 
            y=S @ y_bottom, 
            idx_bottom=idx_bottom
        ),
        S @ y_hat_bottom
    )

<a name="cell-3"></a>
## 3. Min Trace

This reconciliation algorithm proposed by Wickramasuriya et al. depends on a generalized least squares estimator and an estimator of the covariance matrix of the coherency errors $\mathbf{W}_{h}$. The Min Trace algorithm minimizes the squared errors for the coherent forecasts under an unbiasedness assumption; the solution has a closed form.

$$\mathbf{P}_{\text{MinT}} = \left(\mathbf{S}^{\intercal}\mathbf{W}_{h}\mathbf{S}\right)^{-1} \mathbf{S}^{\intercal} \mathbf{W}^{-1}_{h}$$

- [Wickramasuriya, S. L., Athanasopoulos, G., & Hyndman, R. J. (2019). Optimal forecast reconciliation for hierarchical and grouped time series through trace minimization. Journal of the American Statistical Association, 114 , 804–819. doi:10.1080/01621459.2018.1448825.](https://robjhyndman.com/publications/mint/)

In [ ]:
#export
def crossprod(x):
    return x.T @ x

In [ ]:
#export
def min_trace(S: np.ndarray, 
              y_hat: np.ndarray,
              residuals: np.ndarray,
              method: str):
    # shape residuals (obs, n_hiers)
    res_methods = ['wls_var', 'mint_cov', 'mint_shrink']
    if method in res_methods and residuals is None:
        raise ValueError(f"For methods {', '.join(res_methods)} you need to pass residuals")
    n_hiers, n_bottom = S.shape
    if method == 'ols':
        W = np.eye(n_hiers)
    elif method == 'wls_struct':
        W = np.diag(S @ np.ones((n_bottom,)))
    elif method in res_methods:
        n, _ = residuals.shape
        masked_res = np.ma.array(residuals, mask=np.isnan(residuals))
        covm = np.ma.cov(masked_res, rowvar=False, allow_masked=True).data
        if method == 'wls_var':
            W = np.diag(np.diag(covm))
        elif method == 'mint_cov':
            W = covm
        elif method == 'mint_shrink':
            tar = np.diag(np.diag(covm))
            corm = cov2corr(covm)
            xs = np.divide(residuals, np.sqrt(np.diag(covm)))
            xs = xs[~np.isnan(xs).any(axis=1), :]
            v = (1 / (n * (n - 1))) * (crossprod(xs ** 2) - (1 / n) * (crossprod(xs) ** 2))
            np.fill_diagonal(v, 0)
            corapn = cov2corr(tar)
            d = (corm - corapn) ** 2
            lmd = v.sum() / d.sum()
            lmd = max(min(lmd, 1), 0)
            W = lmd * tar + (1 - lmd) * covm
    else:
        raise ValueError(f'Unkown reconciliation method {method}')
    
    eigenvalues, _ = np.linalg.eig(W)
    if any(eigenvalues < 1e-8):
        raise Exception(f'min_trace ({method}) needs covariance matrix to be positive definite.')
        
    R = S.T @ np.linalg.inv(W)
    P = np.linalg.inv(R @ S) @ R
    
    return _reconcile(S, P, W, y_hat)

In [ ]:
#export
class MinTrace:
    
    def __init__(self, method: str):
        self.method = method
        
    def reconcile(self, 
                  S: np.ndarray, 
                  y_hat: np.ndarray,
                  residuals: np.ndarray):
        return min_trace(S=S, y_hat=y_hat, 
                         residuals=residuals,
                         method=self.method)
    
    __call__ = reconcile

In [ ]:
#hide
for method in ['ols', 'wls_struct', 'wls_var', 'mint_shrink']:
    cls_min_trace = MinTrace(method=method)
    test_close(
        cls_min_trace(
            S, 
            S @ y_hat_bottom, 
            np.transpose(S @ resids_bottom)
        ),
        S @ y_hat_bottom
    )
with ExceptionExpected(regex='min_trace (mint_cov)*'):
    cls_min_trace = MinTrace(method='mint_cov')
    cls_min_trace(
        S, 
        S @ y_hat_bottom, 
        np.transpose(S @ resids_bottom)
    )

In [ ]:
#export
def erm(S: np.ndarray,
        y_hat: np.ndarray,
        method: str,
        lambda_reg: float = 1e-2):
    n_hiers, n_bottom = S.shape
    if method == 'exact':
        B = y_hat.T @ S @ np.linalg.inv(S.T @ S).T
        P = B.T @ y_hat.T @ np.linalg.inv(y_hat @ y_hat.T + lambda_reg * np.eye(n_hiers))
    else:
        raise ValueError(f'Unkown reconciliation method {method}')
        
    W = np.eye(n_hiers, dtype=np.float32)
    
    return _reconcile(S, P, W, y_hat)

<a name="cell-4"></a>
## 4. Empirical Risk Minimization

The Empirical Risk Minimization reconciliation strategy relaxes the unbiasedness assumptions from
previous reconciliation methods like MinT and optimizes square errors between the reconciled predictions
and the validation data to obtain an optimal reconciliation matrix P.

The exact solution for $\mathbf{P}$ (`method='exact'`) follows the expression:

$$\mathbf{P}^{*} = \left(\mathbf{S}^{\intercal}\mathbf{S}\right)^{-1}\mathbf{Y}^{\intercal}\hat{\mathbf{Y}}\left(\hat{\mathbf{Y}}\hat{\mathbf{Y}}\right)^{-1}$$

The alternative Lasso regularized $\mathbf{P}$ solution (`method='lasso'`) is useful when the observations of validation data is 
limited or the exact solution has low numerical stability.

$$\mathbf{P}^{*} = \text{argmin}_{\mathbf{P}} ||\mathbf{Y}-\mathbf{S} \mathbf{P} \hat{Y} ||^{2}_{2} + \lambda ||\mathbf{P}-\mathbf{P}_{\text{BU}}||_{1}$$



- [Ben Taieb, S., & Koo, B. (2019). Regularized regression for hierarchical forecasting without unbiasedness conditions. In Proceedings of the 25th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining KDD '19 (p. 1337{1347). New York, NY, USA: Association for Computing Machinery.](https://doi.org/10.1145/3292500.3330976)

In [ ]:
# export
def _solve(a, b, c):
    """Return solution of a quadratic minimization.
    The optimization equation is:
         f(a, b, c) = argmin_w{1/2 * a * w^2 + b * w + c * |w|}
    we get optimal solution w*:
         w* = -(b - sign(b)*c)/a if |b| > c else w* = 0
    REQUIRES: Dimensionality of a and b must be same
    Args:
      a: array
      b: array
      c: array with one element.
    Returns:
      A Tensor w, which is solution for the equation
    """
    w = (c * np.sign(b) - b) / a
    w = (np.abs(b) > c) * w
    return w

class AdamOptimizer:
    """Stochastic gradient descent optimizer with Adam
    
    Note: All default values are from the original Adam paper
    
        Parameters
        ----------
        params : list, length = len(coefs_) + len(intercepts_)
            The concatenated list containing coefs_ and intercepts_ in MLP model.
            Used for initializing velocities and updating params
        learning_rate_init : float, default=0.001
            The initial learning rate used. It controls the step-size in updating
            the weights
        beta_1 : float, default=0.9
            Exponential decay rate for estimates of first moment vector, should be
            in [0, 1)
        beta_2 : float, default=0.999
            Exponential decay rate for estimates of second moment vector, should be
            in [0, 1)
        epsilon : float, default=1e-8
            Value for numerical stability
            
        Attributes
        ----------
        learning_rate : float
            The current learning rate
        t : int
            Timestep
        ms : list, length = len(params)
            First moment vectors
        vs : list, length = len(params)
            Second moment vectors
            
        References
        ----------
        [1] Kingma, Diederik, and Jimmy Ba (2014) "Adam: A method for
            stochastic optimization."

        [2] Sashank J. Reddi, Satyen Kale, Sanjiv Kumar (2019). "On the 
            Convergence of Adam and Beyond.". 
            https://arxiv.org/abs/1904.09237v1
        
        [3] Manzil Zaheer, Sashank Reddi, Devendra Sachan, Satyen Kale, Sanjiv Kumar (2019).
            "Adaptive Methods for Nonconvex Optimization".
            https://papers.nips.cc/paper/2018/hash/90365351ccc7437a1309dc64e4db32a3-Abstract.html
    """

    def __init__(
        self, params, learning_rate_init=0.001, 
        beta_1=0.9, beta_2=0.999, epsilon=1e-8,
        l1_reg=0,
    ):        
        self.learning_rate_init = learning_rate_init
        self.learning_rate = float(learning_rate_init)        

        self.beta_1 = beta_1
        self.beta_2 = beta_2
        self.epsilon = epsilon
        
        self.l1_reg = l1_reg
        
        self.t = 0
        self.ms = [np.zeros_like(param) for param in params]
        self.vs = [np.zeros_like(param) for param in params]
        
    def update_params(self, params, grads):
        """Update parameters with given gradients
        Parameters
        ----------
        params : list of length = len(coefs_) + len(intercepts_)
            The concatenated list containing coefs_ and intercepts_ in MLP
            model. Used for initializing velocities and updating params
        grads : list of length = len(params)
            Containing gradients with respect to coefs_ and intercepts_ in MLP
            model. So length should be aligned with params
        """
        updates = self._get_updates(grads)
        for param, update, per_coord_lr in zip((p for p in params), updates, 
                                               self.per_coord_lrs):
            param += update
            if self.l1_reg>0:
                param = _solve(1, -param, self.l1_reg * per_coord_lr)
        return params  

    def _get_updates(self, grads):
        """Get the values used to update params with given gradients
        Parameters
        ----------
        grads : list, length = len(coefs_) + len(intercepts_)
            Containing gradients with respect to coefs_ and intercepts_ in MLP
            model. So length should be aligned with params
        Returns
        -------
        updates : list, length = len(grads)
            The values to add to params
        """
        self.t += 1
        
        # Yogi, correction
        signs = np.sign(grads**2 - self.vs)
        
        self.ms = [
            self.beta_1 * m + (1 - self.beta_1) * grad
            for m, grad in zip(self.ms, grads)
        ]
        self.vs = [
            #np.maximum(v, self.beta_2 * v + (1 - self.beta_2) * (grad**2))
            v + (1 - self.beta_2) * sign * (grad**2)
            for v, grad, sign in zip(self.vs, grads, signs)
        ]
        
        self.per_coord_lrs = [
            self.learning_rate_init * 
            (np.sqrt(1 - self.beta_2**self.t) / (1 - self.beta_1**self.t)) /
            (np.sqrt(v) + self.epsilon)
            for v in self.vs
        ]
        
        updates = [ -lr * m for m, lr in zip(self.ms, self.per_coord_lrs)]
        
        return updates

In [ ]:
#export
class ERM:
    
    def __init__(self, method: str, 
                 y_valid: np.array, y_valid_hat: np.array,
                 l1_reg: int = 300, learning_rate: float = 1e-1, batch_size: int = 1, 
                 max_iters: int = 6_000, init_zeros: bool = False, random_state: int = 0):
        
        assert method in ['exact', 'lasso'], f'Unkown reconciliation method {method}'
        self.method = method

        # Lasso hyperparameters
        self.l1_reg = l1_reg
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.max_iters = max_iters
        
        self.init_zeros = init_zeros
        self.random_state = random_state

        self.y_valid = y_valid
        self.y_valid_hat = y_valid_hat

    def _exact_erm_matrix(self, S: np.array, Y: np.array, Y_hat: np.array):
        """
        The quality of this closed form solution of the projection matrix depends on having 
        large enough of uncorrelated Y_hat values for it to have a good condition number.
        We offer the alternative of the Frobeniuous optimized alternative.
        """
        n_total, n_bottom = S.shape
        temp1 = np.linalg.inv(S.T @ S + 1e-2*np.eye(n_bottom))
        temp2 = (S.T @ Y) @ Y_hat.T
        temp3 = np.linalg.inv(Y_hat @ Y_hat.T + 1e-1*np.eye(n_total))
        P     = (temp1 @ temp2) @ temp3
        
        # TODO: Write warning on the condition number.
        # print("cond_number(temp1)", np.linalg.cond(temp1))
        # print("cond_number(temp3)", np.linalg.cond(temp3))
        return P

    def _compute_loss_and_gradients(self, S, P, Pbu, y, y_hat):
        # Wrangling data and flatten it
        # [n_batch,n_total,horizon] -> [n_total,n_batch,horizon]
        n_batch,n_total,horizon = y.shape
        y_t = np.transpose(y, (1, 0, 2))
        y_hat_t = np.transpose(y_hat, (1, 0, 2))
        y_t = y_t.reshape(n_total,n_batch*horizon)
        y_hat_t = y_hat_t.reshape(n_total,n_batch*horizon)
        y_prime = S @ P @ y_hat_t
        
        # Compute loss
        main_loss  = np.mean((y_t - y_prime)**2)
        reg_loss   = np.mean(np.abs(P - Pbu))
        loss = main_loss + self.l1_reg * reg_loss
        
        # Compute gradients 
        g1 = (2 * S.T @ y_prime - 2 * S.T @ y_t) @ y_hat_t.T
        g2 = self.l1_reg * np.sign(P-Pbu)
        
        grads = g1 + g2     
        
        return loss, grads

    def _lasso_erm_matrix(self, S: np.array, Y: np.array, Y_hat: np.array):
        """
        This version of the projection matrix optimizes the reconciliation reconstruction
        error, with Frobenius L1 regularization of the projection around the bottom-up matrix.
        """
        
        # Parse hyperparameters and prepare data
        n_total, n_bottom = S.shape
        n_agg = n_total - n_bottom
        
        if len(Y.shape)==2:
            Y = Y[None,:,:]
            Y_hat = Y_hat[None,:,:]
        
        # Initialize P matrix
        Pbu = np.concatenate([np.zeros((n_agg,n_bottom)), 
                              np.eye(n_bottom)], axis=0).T
        if self.init_zeros:
            P = np.zeros(Pbu.shape)
        else:
            P = Pbu.copy()
        
        optimizer = AdamOptimizer(params=P,
                                  beta_1=0.9,
                                  beta_2=0.999,
                                  epsilon=1e-8,
                                  learning_rate_init=self.learning_rate,
                                  l1_reg=self.l1_reg)
        
        # Datasets permutation for stochastic optimization
        # numpy random seed for replicability, and data augmentation
        np.random.seed(self.random_state)
        batch_idxs = np.random.permutation(len(Y)) 
        Y = Y[batch_idxs,:,:]
        Y_hat = Y_hat[batch_idxs,:,:]
        
        idx = 0
        prev_loss = 0.0
        batch_size = self.batch_size
        iterator = tqdm(range(self.max_iters), mininterval=1)
        for step in iterator:            
            
            # Stochastic gradient's sample
            batch_y = Y[idx:idx+batch_size,:,:]
            batch_y_hat = Y_hat[idx:idx+batch_size,:,:]
            idx = (idx + batch_size) % len(Y)
            
            loss, grads = self._compute_loss_and_gradients(S=S, P=P, Pbu=Pbu, 
                                                           y=batch_y, y_hat=batch_y_hat)
            P = optimizer.update_params(params=P, grads=grads)
            iterator.set_description(f'ERM Loss = {np.round(loss,4)}')
            prev_loss = loss
            
        return P
        
    def reconcile(self, S: np.ndarray,
                  y_hat: np.ndarray):

        if self.method=='exact':
            self.P = self._exact_erm_matrix(S=S, Y=self.y_valid, Y_hat=self.y_valid_hat)
        elif self.method=='lasso':
            self.P = self._lasso_erm_matrix(S=S, Y=self.y_valid, Y_hat=self.y_valid_hat)

        y_hat_reconc = S @ self.P @ y_hat

        return y_hat_reconc
    
    __call__ = reconcile